###### 새버전

In [9]:
# m2.datadam_predict_price

# module
import psycopg2
import datetime
import numpy  as np
import pandas as pd

# 기준월
date = '202208'
# dates = pd.date_range('2022-01',periods=7,freq='M').strftime("%Y%m")

In [10]:
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

In [11]:
# PNU별 평당 분양가
sql = f'''
insert into m2.datadam_predict_price (
    with date as (
        select {date}::text as base_ym
    )
    select A.base_ym
         , A.pnu
         , A.emd_cd
         , A.price
         , sysdate::date as create_at
         , sysdate::date as update_at
         , 'du.Park' as work_user
    from (-- PNU별 가치환원법
        select pnu, substring(pnu,1,8) as emd_cd, round((min_price_1 + max_price_1)/2,0) as price, base_ym
        from m2.cremao_price A -- 예측 분양가
        where base_ym = (select base_ym from date)
        ) A
    join m1.pnu B -- 실제 PNU 확인
        on substring(A.pnu, 1, 19) = B.pnu
    join (-- 실제 건물 확인	
        select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
        from m1.bld_rgst
        where base_ym = (select base_ym from date)
    ) C
        on substring(A.pnu, 1, 19) = C.pnu
);
'''
aws_cursor.execute(sql)
aws_conn.commit()

# row count
print("Done")

Done


###### 구버전

In [35]:
### 0. import
import psycopg2
import datetime
import numpy  as np
import pandas as pd


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# PNU별 평당 분양가
sql = '''
with date as (
    select '202208' as base_ym
)
select A.base_ym
	 , A.pnu
	 , A.emd_cd
	 , A.price
	 , sysdate::date as create_at
     , sysdate::date as update_at
     , 'du.Park' as work_user
from (-- PNU별 가치환원법
	select pnu, substring(pnu,1,8) as emd_cd, round((min_price_1 + max_price_1)/2,0) as price, base_ym
	from m2.cremao_price A -- 예측 분양가
	where base_ym = (select base_ym from date)
	) A
join m1.pnu B -- 실제 PNU 확인
	on substring(A.pnu, 1, 19) = B.pnu
join (-- 실제 건물 확인	
	select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
	from m1.bld_rgst
	where base_ym = (select base_ym from date)
) C
	on substring(A.pnu, 1, 19) = C.pnu
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['base_ym','pnu','emd_cd','price','create_at','update_at','work_user']
df['price'] = df['price'].astype(int)



### 2. DB INSERT
table = 'm2.datadam_predict_price'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-25 16:50:20.313237  :  0
2022-11-25 16:50:26.269422  :  10000
2022-11-25 16:50:28.183118  :  20000
2022-11-25 16:50:30.318509  :  30000
2022-11-25 16:50:32.586800  :  40000
2022-11-25 16:50:34.722829  :  50000
2022-11-25 16:50:36.734441  :  60000
2022-11-25 16:50:38.865634  :  70000
2022-11-25 16:50:40.999410  :  80000
2022-11-25 16:50:57.323132  :  90000
2022-11-25 16:50:59.599587  :  100000
2022-11-25 16:51:01.807295  :  110000
2022-11-25 16:51:04.051293  :  120000
2022-11-25 16:51:06.335122  :  130000
2022-11-25 16:51:08.592624  :  140000
2022-11-25 16:51:10.733632  :  150000
2022-11-25 16:51:12.995562  :  160000
2022-11-25 16:51:15.669582  :  170000
2022-11-25 16:51:17.549778  :  180000
2022-11-25 16:51:19.885471  :  190000
2022-11-25 16:51:22.090334  :  200000
2022-11-25 16:51:24.856889  :  210000
2022-11-25 16:51:27.455577  :  220000
2022-11-25 16:51:30.318377  :  230000
2022-11-25 16:51:32.888709  :  240000
2022-11-25 16:51:35.578395  :  250000
2022-11-25 16:51:38.340358

In [36]:
df

,base_ym,pnu,emd_cd,price,create_at,update_at,work_user
0,202208,1111010100100040014,11110101,48699511,2022-11-25,2022-11-25,du.Park
1,202208,1111010100100040020,11110101,44749981,2022-11-25,2022-11-25,du.Park
2,202208,1111010100100080004,11110101,43648228,2022-11-25,2022-11-25,du.Park
3,202208,1111010100100150004,11110101,43192442,2022-11-25,2022-11-25,du.Park
4,202208,1111010100100150005,11110101,49050490,2022-11-25,2022-11-25,du.Park
...,...,...,...,...,...,...,...
1119100,202208,5013032025118440000,50130320,106147293,2022-11-25,2022-11-25,du.Park
1119101,202208,5013032026100170000,50130320,115234007,2022-11-25,2022-11-25,du.Park
1119102,202208,5013032026100370001,50130320,96490999,2022-11-25,2022-11-25,du.Park
1119103,202208,5013032026101680000,50130320,108855314,2022-11-25,2022-11-25,du.Park
